### [Python multiprocessing tutorial](http://zetcode.com/python/multiprocessing/)

In [1]:
import os
import time
import multiprocessing
from multiprocessing import Process

In [2]:
def fun(name):
    print(f"hello {name}")

def main():
    p = Process(target=fun, args=("Peter",))
    p.start()

main()

hello Peter


The `join` method blocks the execution of the `main` process until the process whose join method is called terminates. Without the join method, the main process won't wait until the process gets terminated.

In [3]:
def fun():

    print('starting fun')
    time.sleep(2)
    print('finishing fun')

def main():

    p = Process(target=fun)
    p.start()
    p.join()

print('starting main')
main()
print('finishing main')

starting main
starting fun
finishing fun
finishing main


The `is_alive` method determines if the process is running.

In [4]:
def fun():
    print('calling fun')
    time.sleep(2)

def main():
    print('main fun')
    p = Process(target=fun)
    p.start()
    p.join()
    print(f'Process p is alive: {p.is_alive()}')

main()

main fun
calling fun
Process p is alive: False


The `os.getpid` returns the current process Id, while the `os.getppid` returns the parent's process Id.

In [5]:
def fun():

    print('--------------------------')
    print('calling fun')
    print('parent process id:', os.getppid())
    print('process id:', os.getpid())

def main():

    print('main fun')
    print('process id:', os.getpid())

    p1 = Process(target=fun)
    p1.start()
    p1.join()

    p2 = Process(target=fun)
    p2.start()
    p2.join()
main()

main fun
process id: 18684
--------------------------
calling fun
parent process id: 18684
process id: 18708
--------------------------
calling fun
parent process id: 18684
process id: 18717


#### Subclassing Process
- When we subclass the Process, we override the run method.

In [7]:
class Worker(Process):

    def run(self):
        print(f'In {self.name}')
        time.sleep(2)

def main():

    worker = Worker()
    worker.start()

    worker2 = Worker()
    worker2.start()

    worker.join()
    worker2.join()

main()

In Worker-9
In Worker-10


### Python multiprocessing Pool
- The management of the worker processes can be simplified with the Pool object. It controls a pool of worker processes to which jobs can be submitted. The pool's map method chops the given iterable into a number of chunks which it submits to the process pool as separate tasks. The pool's map is a parallel equivalent of the built-in map method. The map blocks the main execution until all computations finish.

- The Pool can take the number of processes as a parameter. It is a value with which we can experiment. If we do not provide any value, then the number returned by `os.cpu_count()` is used.

### Python multiprocessing Pool


In [8]:
from timeit import default_timer as timer
from multiprocessing import Pool, cpu_count

In [9]:
def square(n):

    time.sleep(2)
    return n * n

def main():

    start = timer()
    print(f'starting computations on {cpu_count()} cores')
    values = (2, 4, 6, 8)
    with Pool() as pool:
        res = pool.map(square, values)
        print(res)

    end = timer()
    print(f'elapsed time: {end - start}')

main()

starting computations on 8 cores
[4, 16, 36, 64]
elapsed time: 2.092155788999662


### Multiple arguments


In [10]:
from multiprocessing import Pool
import functools

def inc(x):
    return x + 1

def dec(x):
    return x - 1

def add(x, y):
    return x + y

def smap(f):
    return f()

def main():

    f_inc = functools.partial(inc, 4)
    f_dec = functools.partial(dec, 2)
    f_add = functools.partial(add, 3, 4)

    with Pool() as pool:
        res = pool.map(smap, [f_inc, f_dec, f_add])
        print(res)

main()

[5, 1, 7]


### Python multiprocessing π calculation

In [12]:
from decimal import Decimal, getcontext
from timeit import default_timer as timer

def pi(precision):

    getcontext().prec = precision

    return sum(1/Decimal(16)**k *
        (Decimal(4)/(8*k+1) -
         Decimal(2)/(8*k+4) -
         Decimal(1)/(8*k+5) -
         Decimal(1)/(8*k+6)) for k in range (precision))


start = timer()
values = (1000, 1500, 2000)
data = list(map(pi, values))
# print(data)

end = timer()
print(f'sequentially: {end - start}')

sequentially: 0.4356065590000071


In [14]:
from decimal import Decimal, getcontext
from timeit import default_timer as timer
from multiprocessing import Pool, current_process
import time


def pi(precision):

    getcontext().prec=precision

    return sum(1/Decimal(16)**k *
        (Decimal(4)/(8*k+1) -
         Decimal(2)/(8*k+4) -
         Decimal(1)/(8*k+5) -
         Decimal(1)/(8*k+6)) for k in range (precision))


def main():

    start = timer()

    with Pool(3) as pool:

        values = (1000, 1500, 2000)
        data = pool.map(pi, values)
#         print(data)

    end = timer()
    print(f'paralelly: {end - start}')

main()

paralelly: 0.3123783029986953


### Separate memory in a process


In [15]:
from multiprocessing import Process, current_process

data = [1, 2]

def fun():

    global data
    data.extend((3, 4, 5))
    print(f'Result in {current_process().name}: {data}')

def main():
    worker = Process(target=fun)
    worker.start()
    worker.join()

    print(f'Result in main: {data}')

main()

Result in Process-33: [1, 2, 3, 4, 5]
Result in main: [1, 2]
